In [21]:
import cv2
import random
import numpy as np
import math
import os
import statistics
from PIL import Image, ImageOps 
import matplotlib.pyplot as plt
from collections import Counter
from scipy.spatial import distance

In [22]:
img_read_1 = Image.open(r"Q3_img_back_1.jpg") 
img_read_1 = img_read_1.resize((256, 256))
img_array__rgb_1 = np.array(img_read_1)
img_gray_1 = ImageOps.grayscale(img_read_1)
img_array_1 = np.array(img_gray_1)

In [23]:
#Preparing a numpy array of size of image containing the LBP values of pixels
temp_pad_1 = np.pad(img_array_1, pad_width=1, mode='constant', constant_values=0)
no_rows = temp_pad_1.shape[0]
no_col = temp_pad_1.shape[1]
list_lbp = []
for i in range(1, no_rows-1):
    list_temp = []
    for j in range(1, no_col-1):
        curr_num = temp_pad_1[i][j]
        list_nums = [temp_pad_1[i+1][j-1],temp_pad_1[i][j-1],temp_pad_1[i-1][j-1],temp_pad_1[i-1][j],
                     temp_pad_1[i-1][j+1],temp_pad_1[i][j+1],temp_pad_1[i+1][j+1],temp_pad_1[i+1][j]]
        temp_str = ''
        for k in list_nums:
            if k < curr_num:
                temp_str = temp_str + '0'
            else:
                temp_str = temp_str + '1'
        
        ctr = 0
        num_dec = 0
        for m in range(len(temp_str)-1, -1, -1):
            if temp_str[m] == '0':
                num_dec += 0
            else:
                num_dec = num_dec + (2 ** ctr)
            ctr += 1
        
        list_temp.append(num_dec)
    list_lbp.append(list_temp)

np_lbp_1 = np.array(list_lbp)

In [24]:
#Now picking patches of size 9 X 9
patch_rows = 16
patch_cols = 16

end_row = np_lbp_1.shape[0] - patch_rows 
end_col = np_lbp_1.shape[1] - patch_cols
dict_patches_histogram_1 = {}
patch_id = 0
for a in range(0, end_row + 1, patch_rows):
    slice_till_row = a + patch_rows - 1
    for b in range(0, end_col + 1, patch_cols):
        patch_id += 1
        list_lbp_vals = []
        slice_till_col = b + patch_cols - 1
        temp_np_array = np_lbp_1[a:slice_till_row+1, b:slice_till_col+1]
        for c in temp_np_array:
            for d in range(len(c)):
                list_lbp_vals.append(c[d])
        
        patch_histogram = Counter(list_lbp_vals)
        temp_sum = 0
        for x in patch_histogram:
            temp_sum += patch_histogram[x]
        for y in patch_histogram:
            patch_histogram[y] = patch_histogram[y] / temp_sum
        dict_patches_histogram_1[patch_id] = patch_histogram

In [25]:
#Now doing all tasks for image 2
img_read_2 = Image.open(r"Q3_img_back_2.png") 
img_read_2 = img_read_2.resize((256, 256))
img_gray_2 = ImageOps.grayscale(img_read_2)
img_array_2 = np.array(img_gray_2)

In [26]:
#Preparing a numpy array of size of image containing the LBP values of pixels
temp_pad_2 = np.pad(img_array_2, pad_width=1, mode='constant', constant_values=0)
no_rows = temp_pad_2.shape[0]
no_col = temp_pad_2.shape[1]
list_lbp = []
for i in range(1, no_rows-1):
    list_temp = []
    for j in range(1, no_col-1):
        curr_num = temp_pad_2[i][j]
        list_nums = [temp_pad_2[i+1][j-1],temp_pad_2[i][j-1],temp_pad_2[i-1][j-1],temp_pad_2[i-1][j],
                     temp_pad_2[i-1][j+1],temp_pad_2[i][j+1],temp_pad_2[i+1][j+1],temp_pad_2[i+1][j]]
        temp_str = ''
        for k in list_nums:
            if k < curr_num:
                temp_str = temp_str + '0'
            else:
                temp_str = temp_str + '1'
        
        ctr = 0
        num_dec = 0
        for m in range(len(temp_str)-1, -1, -1):
            if temp_str[m] == '0':
                num_dec += 0
            else:
                num_dec = num_dec + (2 ** ctr)
            ctr += 1
        
        list_temp.append(num_dec)
    list_lbp.append(list_temp)

np_lbp_2 = np.array(list_lbp)

In [27]:
#Now picking patches of size 9 X 9

end_row = np_lbp_2.shape[0] - patch_rows 
end_col = np_lbp_2.shape[1] - patch_cols
dict_patches_histogram_2 = {}
patch_id = 0
for a in range(0, end_row + 1, patch_rows):
    slice_till_row = a + patch_rows - 1
    for b in range(0, end_col + 1, patch_cols):
        patch_id += 1
        list_lbp_vals = []
        slice_till_col = b + patch_cols - 1
        temp_np_array = np_lbp_2[a:slice_till_row+1, b:slice_till_col+1]
        for c in temp_np_array:
            for d in range(len(c)):
                list_lbp_vals.append(c[d])
        
        patch_histogram = Counter(list_lbp_vals)
        temp_sum = 0
        for x in patch_histogram:
            temp_sum += patch_histogram[x]
        for y in patch_histogram:
            patch_histogram[y] = patch_histogram[y] / temp_sum
        dict_patches_histogram_2[patch_id] = patch_histogram


In [28]:
from tqdm import tqdm
#Now finding the chi-square distance between the hoistograms of patches
dict_matching_patches = {}

for p_id_1 in dict_patches_histogram_1:
    curr_histogram_1 = dict_patches_histogram_1[p_id_1]
    for p_id_2 in dict_patches_histogram_2:
        curr_histogram_2 = dict_patches_histogram_2[p_id_2]
        chi_square_dist = 0
        for lbp_val in range(0,256):
            val_xi = curr_histogram_1[lbp_val]
            val_yi = curr_histogram_2[lbp_val]
            numerator_val = (val_xi - val_yi) ** 2
            denominator_val = (val_xi + val_yi)
            if denominator_val == 0:
                chi_square_dist += 0
            else:
                chi_square_dist = chi_square_dist + (numerator_val / denominator_val)
    
    if dict_matching_patches.get(p_id_1) is None:
        dict_matching_patches[p_id_1] = [p_id_2]
    else:
        dict_matching_patches[p_id_1].append(p_id_2)

In [29]:
dir_matching_patches = 'Patches\\'
end_row = np_lbp_2.shape[0] - patch_rows 
end_col = np_lbp_2.shape[1] - patch_cols
dict_patches_histogram_2 = {}
patch_id = 0
for a in range(0, end_row + 1, patch_rows):
    slice_till_row = a + patch_rows - 1
    for b in range(0, end_col + 1, patch_cols):
        patch_id += 1
        list_lbp_vals = []
        slice_till_col = b + patch_cols - 1
        temp_np_array = img_array__rgb_1[a:slice_till_row+1, b:slice_till_col+1, :]
        if dict_matching_patches.get(patch_id) is not None:
            img_flag = Image.fromarray(temp_np_array)
            xc = 'patch'
            xt = xc + str(patch_id) + '.png'
            comp_path = os.path.join(dir_matching_patches, xt)
            img_flag.save(comp_path)